In [ ]:
import inversion_utils as iu
import numpy as np
import importlib
importlib.reload(iu)
import remap
importlib.reload(remap)
from astropy.coordinates import SkyCoord
import astropy.units as u
import me_utils as meu
from sunpy.coordinates import frames
from sunpy.map.header_helper import make_fitswcs_header
from sunpy.map import Map
from astropy.io import fits
import matplotlib.pyplot as plt
import scipy.ndimage
from scipy.io import readsav


In [ ]:
data_dir = '/mn/stornext/d20/RoCS/aditi/project/sst/inversion/pyMilne/ITA/temp3/'
# fblos = 'Blos_6173_2021-06-22_T081750_2021-06-22_T090813_t_0-162.fcube'
# fbhor = 'Bhor_6173_2021-06-22_T081750_2021-06-22_T090813_t_0-162.fcube'
# fbazi = 'Bazi_6173_2021-06-22_T081750_2021-06-22_T090813_t_0-162.fcube'
disambig_sav = '/mn/stornext/d20/RoCS/aditi/project/sst/inversion/ambig/test/QS/ambig/22_06_2021_h0817/QS_22_06_2021_h0817_nb_6173_ambig_1721211858_1278_1267_disambig_Bxyz.sav'

In [ ]:
data_dir = '/mn/stornext/d18/lapalma/reduc/2021/2021-06-22/CRISP/cubes_nb/'
crisp_im = data_dir + 'nb_6173_2021-06-22T08:17:48_scans=0-162_stokes_corrected_im.fits'
fits_header = iu.load_fits_header(crisp_im, out_dict=True)
fits_info = iu.get_fits_info(crisp_im)
crval1 = fits_info['xcent0']
crval2 = fits_info['ycent0']
rescale = 1.0
cdelt1 = 0.058 * rescale 
crota2 = 180.0
date_obs = fits_info['avg_time_obs']

In [ ]:
disambig = readsav(disambig_sav)
bx0 = disambig['bx0']
by0 = disambig['by0']
bz0 = disambig['bz0']

iu.plot_images([bx0, by0, bz0], title=['Bx', 'By', 'Bz'], cmap='gray', vmin=[-200,-200,-100], vmax=[200,200,100])

In [ ]:
bx1 = scipy.ndimage.rotate(bx0, crota2, reshape=True)
by1 = scipy.ndimage.rotate(by0, crota2, reshape=True)
bz1 = scipy.ndimage.rotate(bz0, crota2, reshape=True)

In [ ]:
bz1_map = remap.make_map(bz1, crval1, crval2, cdelt1, crota2, date_obs, verbose=False)
bz1_map.peek(vmin=-100, vmax=100)

In [ ]:
x1, x2 = -12, 0 # arcsec
y1, y2 = 840, 845   # arcsec
bz0_map = remap.make_map(bz0, crval1, crval2, cdelt1, 0, date_obs, verbose=False)
bottom_left = SkyCoord(x1 * u.arcsec, y1 * u.arcsec, frame=bz0_map.coordinate_frame)
top_right = SkyCoord(x2 * u.arcsec, y2* u.arcsec, frame=bz0_map.coordinate_frame)
bz0_submap = bz0_map.submap(bottom_left, top_right=top_right)
bz0_submap.peek(vmin=-100, vmax=100)


In [ ]:
remap.plot_map_on_grid(bz1_map, vmin_percentile=0.5, vmax_percentile=99.5)

In [ ]:
convert = bz0_submap.center.transform_to(frames.Heliocentric)
xc1, yc1, zc1 = convert.x, convert.y, convert.z

In [ ]:
import numpy as np

def normalize_vector(vector):
    """
    Normalize a 3D vector to get a unit vector.

    Parameters:
    vector (np.ndarray): The input vector to be normalized.

    Returns:
    np.ndarray: The normalized unit vector.
    """
    magnitude = np.linalg.norm(vector)
    if magnitude == 0:
        raise ValueError("Cannot normalize a zero vector")
    unit_vector = vector / magnitude
    return unit_vector

# Example usage:
# vector = np.array([3, 4, 5])
unit_vector = normalize_vector([xc1.value, yc1.value, zc1.value])
print("Unit vector:", unit_vector)


In [ ]:
import numpy as np

def normalize_vector(vector):
    """
    Normalize a 3D vector to get a unit vector.

    Parameters:
    vector (np.ndarray): The input vector to be normalized.

    Returns:
    np.ndarray: The normalized unit vector.
    """
    magnitude = np.linalg.norm(vector)
    if magnitude == 0:
        raise ValueError("Cannot normalize a zero vector")
    unit_vector = vector / magnitude
    return unit_vector

def inward_vector(vector):
    """
    Get the inward-pointing unit vector for a given outward-pointing vector.

    Parameters:
    vector (np.ndarray): The input vector pointing outwards.

    Returns:
    np.ndarray: The normalized unit vector pointing inwards.
    """
    outward_unit_vector = normalize_vector(vector)
    inward_unit_vector = -outward_unit_vector
    return inward_unit_vector

# Example usage:
# vector = np.array([3, 4, 5])
vector = [xc1.value, yc1.value, zc1.value]
outward_unit_vector = normalize_vector(vector)
inward_unit_vector = inward_vector(vector)

print("Outward unit vector:", outward_unit_vector)
print("Inward unit vector:", inward_unit_vector)


In [ ]:
wcs_dict = remap.get_wcs_info(bz1_map, verbose=True)

In [ ]:
bz_cea = remap.remap2cea(wcs_dict, bz1, debug=False)
bz_cea.peek(vmin=-100, vmax=100)

In [ ]:
bxc, byc, bzc, header = remap.bvec2cea(wcs_dict, bx1, by1, bz1, debug=False)
bzc_map = Map(bzc, header)
bxc_map = Map(bxc, header)
byc_map = Map(byc, header)

In [ ]:
iu.plot_images([bxc, byc, bzc], title=['Bx', 'By', 'Bz'], grid_shape=(1, 3), cmap='gray', figsize=(16, 7), aspect='auto', vmin=[-200,-200,-100], vmax=[200,200,100])

---

In [ ]:
crisp_im =  "/mn/stornext/d18/lapalma/reduc/2020/2020-08-07/CRISP/cubes_nb/nb_6173_2020-08-07T08:22:14_scans=0-56_stokes_corrected_im.fits"
fits_header = iu.load_fits_header(crisp_im, out_dict=True)
fits_info = iu.get_fits_info(crisp_im)

In [ ]:
fits_info['xcent0'], fits_info['ycent0']

In [ ]:
crval1 = -448
crval2 = 283
rescale = 2.0
cdelt1 = 0.058 * rescale 
crota2 = 180.0
date_obs = fits_header['DATE-OBS']

In [ ]:
save_dir = '/mn/stornext/d9/data/avijeetp/1_Projects/61_QSEB_Topology/2020-08-07/'
bxfile = save_dir + '6173_2020-08-07_T083019_2020-08-07_T083019_1721136823_sfq_bx.npy'
byfile = save_dir + '6173_2020-08-07_T083019_2020-08-07_T083019_1721136823_sfq_by.npy'
bzfile = save_dir + '6173_2020-08-07_T083019_2020-08-07_T083019_1721136823_sfq_bz.npy'

In [ ]:
bx0 = np.load(bxfile)
by0 = np.load(byfile)
bz0 = np.load(bzfile)

In [ ]:
iu.plot_images([bx0, by0, bz0], title=['Bx', 'By','Bz'], fontsize=10, figsize=(12,5), cmap='seismic', grid_shape=(1, 3) )

In [ ]:
fov=0
bx1 = iu.make_north_up(bx0, fov).T
by1 = iu.make_north_up(by0, fov).T
bz1 = iu.make_north_up(bz0, fov).T

bx2 = -by1 # Because remap2cea expects -bx, also bx, by could be swapped here
by2 = bx1
bz2 = bz1
iu.plot_images([bx2, by2, bz2], title=['Bx', 'By','Bz'], fontsize=10, figsize=(12,5), cmap='seismic', grid_shape=(1, 3), aspect='equal' )

In [ ]:
# data = np.nan_to_num(data)
bx3 = scipy.ndimage.rotate(bx2, crota2, reshape=True)
by3 = scipy.ndimage.rotate(by2, crota2, reshape=True)
bz3 = scipy.ndimage.rotate(bz2, crota2, reshape=True)

In [ ]:
iu.plot_images([bx3, by3, bz3], title=['Bx', 'By','Bz'], fontsize=10, figsize=(12,5), cmap='seismic', grid_shape=(1, 3) )

In [ ]:
bz3_map = remap.make_map(bz3, crval1, crval2, cdelt1, crota2, date_obs, verbose=False)
bz3_map.peek()

In [ ]:
remap.plot_map_on_grid(bz3_map, vmin_percentile=0.5, vmax_percentile=99.5, project_dc=True)

In [ ]:
wcs_dict = remap.get_wcs_info(bz3_map, verbose=True)

In [ ]:
importlib.reload(remap)
bz_cea = remap.remap2cea(wcs_dict, bz3, debug=False)
bz_cea.peek()

In [ ]:

# peff, lat_it, lon_it, latc, bz_cea = remap.remap2cea(wcs_dict, bz1.T, deltal=deltal, debug=False)
# Fx -> -Bx with 180 deg rotation
# Fy -> By with 180 deg rotation
# Fz -> Bz with 180 deg rotation
bxc, byc, bzc, header = remap.bvec2cea(wcs_dict, bx3, by3, bz3, debug=False)

In [ ]:
bzc_map = Map(bzc, header)
bxc_map = Map(bxc, header)
byc_map = Map(byc, header)

In [ ]:
iu.plot_images([bxc, byc, bzc], title=['Bx', 'By', 'Bz'], grid_shape=(1, 3), cmap='seismic', figsize=(16, 7), aspect='auto')

In [ ]:
iu.plot_images([bx3, by3, bz3, bxc, byc, bzc], title=['Fx', 'Fy', 'Fz', 'Bx', 'By', 'Bz'], grid_shape=(2, 3), cmap='seismic', figsize=(15, 11), aspect='auto')

---

In [ ]:
import inversion_utils as iu
import numpy as np
import importlib
importlib.reload(iu)
import remap
importlib.reload(remap)
from astropy.coordinates import SkyCoord
import astropy.units as u
import me_utils as meu
from sunpy.coordinates import frames
from sunpy.map.header_helper import make_fitswcs_header
from sunpy.map import Map
from astropy.io import fits
import matplotlib.pyplot as plt
import scipy.ndimage
from scipy.io import readsav


In [ ]:
crisp_im =  "/mn/stornext/d12/lapalma/reduc/2023/2023-07-26/CRISP/cubes_nb/nb_6173_2023-07-26T08:28:57_08:28:57=0-58_stokes_corrected_im.fits"
fits_header = iu.load_fits_header(crisp_im, out_dict=True)
fits_info = iu.get_fits_info(crisp_im)
fits_info['xcent0'], fits_info['ycent0']

In [ ]:
crval1 = -633.92
crval2 = -211.48
rescale = 1.0
cdelt1 = 0.044 * rescale 
crota2 = 180.0


In [ ]:
date_obs = fits_header['DATE-OBS']
save_dir = '/mn/stornext/d9/data/avijeetp/1_Projects/33_SST/2023-07-26/'

In [ ]:
# load the disambig output file and extract the variables
disambig_out = save_dir+'_1724921179_disambig_output_35_full_res.npz'
data = np.load(disambig_out)
bx0 = data['bx']
by0 = data['by']
bz0 = data['bz']
# # bazi = data['bazi']

In [ ]:
# bxfile = save_dir + '6173_2020-08-07_T083019_2020-08-07_T083019_1721136823_sfq_bx.npy'
# byfile = save_dir + '6173_2020-08-07_T083019_2020-08-07_T083019_1721136823_sfq_by.npy'
# bzfile = save_dir + '6173_2020-08-07_T083019_2020-08-07_T083019_1721136823_sfq_bz.npy'
# bx0 = np.load(bxfile)
# by0 = np.load(byfile)
# bz0 = np.load(bzfile)
iu.plot_images([bx0, by0, bz0], title=['Bx', 'By','Bz'], fontsize=10, figsize=(12,5), cmap='seismic', grid_shape=(1, 3) )


In [ ]:
fov=0
# bx1 = iu.make_north_up(bx0, fov)
# by1 = iu.make_north_up(by0, fov)
# bz1 = iu.make_north_up(bz0, fov)
bx1 = bx0
by1 = by0
bz1 = bz0

In [ ]:

bx2 = -by1 # Because remap2cea expects -bx, also bx, by could be swapped here
by2 = bx1
bz2 = bz1
iu.plot_images([bx2, by2, bz2], title=['Bx', 'By','Bz'], fontsize=10, figsize=(12,5), cmap='seismic', grid_shape=(1, 3), aspect='equal' )


In [ ]:
# data = np.nan_to_num(data)
bx3 = scipy.ndimage.rotate(bx2, crota2, reshape=True)
by3 = scipy.ndimage.rotate(by2, crota2, reshape=True)
bz3 = scipy.ndimage.rotate(bz2, crota2, reshape=True)
iu.plot_images([bx3, by3, bz3], title=['Bx', 'By','Bz'], fontsize=10, figsize=(12,5), cmap='seismic', grid_shape=(1, 3) )
bz3_map = remap.make_map(bz3, crval1, crval2, cdelt1, crota2, date_obs, verbose=False)
bz3_map.peek()


In [ ]:
remap.plot_map_on_grid(bz3_map, vmin_percentile=0.5, vmax_percentile=99.5, project_dc=True)
wcs_dict = remap.get_wcs_info(bz3_map, verbose=True)
importlib.reload(remap)
bz_cea = remap.remap2cea(wcs_dict, bz3, debug=False)
bz_cea.peek()


In [ ]:

# peff, lat_it, lon_it, latc, bz_cea = remap.remap2cea(wcs_dict, bz1.T, deltal=deltal, debug=False)
# Fx -> -Bx with 180 deg rotation
# Fy -> By with 180 deg rotation
# Fz -> Bz with 180 deg rotation
bxc, byc, bzc, header = remap.bvec2cea(wcs_dict, bx3, by3, bz3, debug=False)
bzc_map = Map(bzc, header)
bxc_map = Map(bxc, header)
byc_map = Map(byc, header)


In [ ]:
iu.plot_images([bxc, byc, bzc], title=['Bx', 'By', 'Bz'], grid_shape=(1, 3), cmap='gray', figsize=(16, 7), aspect=1, interpolation='gaussian', vmin=-1500, vmax=1500)
# iu.plot_images([bx3, by3, bz3, bxc, byc, bzc], title=['Fx', 'Fy', 'Fz', 'Bx', 'By', 'Bz'], grid_shape=(2, 3), cmap='seismic', figsize=(15, 11), aspect='auto')

In [ ]:
# save the bxc, byc, bzc maps to fits files
from astropy.io.fits import CompImageHDU
bxc_map.save(save_dir + "bx_cea_35.fits", hdu_type=CompImageHDU, overwrite=True)
byc_map.save(save_dir + "by_cea_35.fits", hdu_type=CompImageHDU, overwrite=True)
bzc_map.save(save_dir + "bz_cea_35.fits", hdu_type=CompImageHDU, overwrite=True)
print(f"saved file: {save_dir}bx_cea_35.fits")
print(f"saved file: {save_dir}by_cea_35.fits")
print(f"saved file: {save_dir}bz_cea_35.fits")

In [ ]:
bxc_map.data.shape